# [STARTER] Udaplay Project

## Part 02 - Agent

In this part of the project, you'll use your VectorDB to be part of your Agent as a tool.

You're building UdaPlay, an AI Research Agent for the video game industry. The agent will:
1. Answer questions using internal knowledge (RAG)
2. Search the web when needed
3. Maintain conversation state
4. Return structured outputs
5. Store useful information for future use

### Setup

In [15]:
# Only needed for Udacity workspace

import importlib.util
import sys

# Check if 'pysqlite3' is available before importing
if importlib.util.find_spec("pysqlite3") is not None:
    import pysqlite3
    sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')

In [7]:
import os
import json
import chromadb
from typing import TypedDict, List, Dict, Optional, Union
from pydantic import BaseModel, Field
from tavily import TavilyClient
from datetime import datetime
from chromadb.utils import embedding_functions

from lib.state_machine import StateMachine, Step, EntryPoint, Termination, Run, Resource
from lib.llm import LLM
from lib.messages import UserMessage, SystemMessage, AIMessage
from lib.tooling import tool
from lib.vector_db import VectorStoreManager, VectorStore
from lib.rag import RAG
from lib.memory import LongTermMemory, MemoryFragment, MemorySearchResult
from lib.parsers import PydanticOutputParser
from dotenv import load_dotenv

In [2]:
load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")

# Direct ChromaDB connection (bypassing VectorStoreManager.get_store issue)
chroma_client = chromadb.PersistentClient(path="chromadb")

# Create embedding function
embedding_fn = embedding_functions.OpenAIEmbeddingFunction(
    api_key=OPENAI_API_KEY
)

# Get collection directly
collection = chroma_client.get_collection(
    name="udaplay",
    embedding_function=embedding_fn
)
print(f"Loaded {collection.count()} games from database")

vector_store = VectorStore(collection)

# Create Vector Manager for long-term memory
Vector_Manager = VectorStoreManager(openai_api_key=OPENAI_API_KEY)
long_term_memory = LongTermMemory(Vector_Manager)

# Create RAG
games_rag = RAG(
    llm=LLM(model="gpt-4o-mini"),
    vector_store=vector_store
)

# Tavily client
tavily_client = TavilyClient(api_key=TAVILY_API_KEY)

print("All services initialized!")

Loaded 15 games from database
All services initialized!


In [3]:
# Cell: Test Everything Works
print("🧪 Testing all services...")
print("=" * 60)

# Test 1: Vector Store
print("1️⃣ Vector Store:")
print(f"   Documents: {vector_store._collection.count()}")

test_query = vector_store.query(query_texts=["Mario"], n_results=1)
print(f"   Query test: ✅ {test_query['metadatas'][0][0].get('Name')}")

# Test 2: RAG
print("\n2️⃣ RAG System:")
rag_result = games_rag.invoke("What is Super Mario 64?")
answer = rag_result.get_final_state()["answer"]
print(f"   RAG test: ✅ (Answer: {answer[:100]}...)")

# Test 3: Long-term Memory
print("\n3️⃣ Long-term Memory:")
print(f"   Status: ✅ Initialized")

# Test 4: Tavily
print("\n4️⃣ Tavily Client:")
print(f"   Status: ✅ Initialized")

print("=" * 60)
print("✅ ALL TESTS PASSED! Ready to build the agent!")

🧪 Testing all services...
1️⃣ Vector Store:
   Documents: 15
   Query test: ✅ Super Mario 64

2️⃣ RAG System:
[StateMachine] Starting: __entry__
[StateMachine] Executing step: retrieve
[StateMachine] Executing step: augment
[StateMachine] Executing step: generate
[StateMachine] Terminating: __termination__
   RAG test: ✅ (Answer: Super Mario 64 is a groundbreaking 3D platformer released in 1996 for the Nintendo 64. It set new st...)

3️⃣ Long-term Memory:
   Status: ✅ Initialized

4️⃣ Tavily Client:
   Status: ✅ Initialized
✅ ALL TESTS PASSED! Ready to build the agent!


In [4]:
class UdaPlayState(TypedDict):
    """State schema for UdaPlay agent workflow"""
    # Input
    user_id: str                        
    user_query: str                       
    
    # Memory
    retrieved_memories: List[str]         
    
    # Search Results
    rag_results: Optional[str]           
    rag_documents: Optional[List[str]]    
    rag_distances: Optional[List[float]]  
    # Evaluation
    results_are_good: bool                
    evaluation_reason: str               
    
    # Web Search (fallback)
    web_search_results: Optional[str]     
    
    # Final Output
    final_answer: str                     
    facts_to_remember: List[str]          
    
    # Metadata
    total_tokens: int                    

In [28]:
class ResultEvaluation(BaseModel):
    """Evaluation of search results quality"""
    is_sufficient: bool = Field(description="Wheter results can answer the qquestion")
    confidence: float = Field(description="Confidence score 0-1", ge=0, le=1)
    reason: str = Field(description="Explanation fo the evaluation")

class AnswerwithMemory(BaseModel):
    """Final answer with facts to remember"""
    
    answer: str = Field(description="The complete anwer to the user's question")
    facts_to_remember: List[str]= Field(
        description="important fact about the user to store, in a long memory",
        default_factory=list
    )

class MemorySummary(BaseModel):
    """Summary of relevant user memories"""
    relvant_fact: List[str] = Field(description="Relevant facts from user history")
    should_personalize: bool = Field(description="Whether to personalize the response")


### Tools

Build at least 3 tools:
- retrieve_game: To search the vector DB
- evaluate_retrieval: To assess the retrieval performance
- game_web_search: If no good, search the web


In [10]:
def retrieve_user_memories_step(state:UdaPlayState, resource: Resource) -> Dict:
    """ Retrieve relevant user memories from long-term storage"""
    user_id= state["user_id"]
    query= state["user_query"]
    ltm: LongTermMemory = resource.vars["long_term_memory"]

    memory_results: MemorySearchResult = ltm.search(
        query_text=query,
        owner=user_id,
        limit=5,
        namespace="preferences"
    )
    retrieved_memories = [frag.content for frag in memory_results.fragments]

    print(f"[Memory] Retrieved {len(retrieved_memories)} memories for user {user_id}")
    
    return {
        "retrieved_memories": retrieved_memories
    }

#### Retrieve Game Tool

In [ ]:
def rag_search_step(state: UdaPlayState, resurce: Resource) -> Dict:
    query = state["user_query"]
    memories = state.get("retrieved_memories", [])
    rag: RAG = resurce.vars["rag"]

    enhanced_query = query

    if memories:
        context= " ".join(memories[:5])
        enhanced_query = f"{query}\n\nUser context: {context}"

    rag_run = rag.invoke(enhanced_query)
    rag_final_state = rag_run.get_final_state()
    
    print(f"[RAG] Searched game database")
    print(f"[RAG] Retrieved {len(rag_final_state.get('documents', []))} documents")
    
    return {
        "rag_results": rag_final_state.get("answer", ""),
        "rag_documents": rag_final_state.get("documents", []),
        "rag_distances": rag_final_state.get("distances", [])
    }

#### Evaluate Retrieval Tool

In [16]:
def evaluate_results_step(state: UdaPlayState, resource: Resource) -> Dict:
    query = state["user_query"]
    rag_answer= state.get("rag_results", [])
    documents = state.get("rage_documents", [])
    llm: LLM = resource.vars["llm"]


    prompt = f"""Evaluate if the retrieved information is sufficient to answer the user's question.

User Question: {query}

Retrieved Answer: {rag_answer}

Number of source documents: {len(documents)}

Evaluate:
1. Does the answer directly address the question?
2. Is the information complete and accurate?
3. Rate your confidence (0-1)

Provide your evaluation."""


    response = llm.invoke(
        input = prompt,
        response_format= ResultEvaluation
    )

    parser = PydanticOutputParser(model_class=ResultEvaluation)
    evaluation= parser.parse(response)

    print(f"[Eval] Sufficient: {evaluation.is_sufficient} (confidence: {evaluation.confidence})")
    print(f"[Eval] Reason: {evaluation.reason}")
    
    return {
        "results_are_good": evaluation.is_sufficient,
        "evaluation_reason": evaluation.reason
    }

#### Game Web Search Tool

In [17]:
def web_search_step(state: UdaPlayState, resource: Resource) -> Dict:
    
    query= state["user_query"]
    tavily: TavilyClient = resource.vars["tavily"]

    print(f"[Web] RAG results insufficient, searching web...")

    try:
        search_results = tavily.search(
            query=f"{query} video game",
            max_results=3,
            search_depth="basic"
        )
        
        # Format results
        formatted = []
        for result in search_results.get('results', []):
            formatted.append({
                "title": result.get('title', ''),
                "content": result.get('content', '')[:500],  # Truncate
                "url": result.get('url', '')
            })
        
        web_results = json.dumps(formatted, indent=2)
        print(f"[Web] Found {len(formatted)} results")
        
    except Exception as e:
        print(f"[Web] Error: {e}")
        web_results = json.dumps({"error": str(e)})
    
    return {
        "web_search_results": web_results
    }

In [18]:
def generate_answer_step(state: UdaPlayState, resource: Resource) -> Dict:

    query = state["user_query"]
    rag_answer = state.get("rag_results", "")
    web_results = state.get("web_search_results")
    memories = state.get("retrieved_memories", [])
    results_good = state.get("results_are_good", False)
    llm: LLM = resource.vars["llm"]
    
    # Build context
    context_parts = []
    
    if results_good and rag_answer:
        context_parts.append(f"Game Database Answer:\n{rag_answer}")
    
    if web_results:
        context_parts.append(f"Web Search Results:\n{web_results}")
    
    if memories:
        context_parts.append(f"User Preferences:\n" + "\n".join(f"- {m}" for m in memories[:3]))
    
    context = "\n\n".join(context_parts)
    
    # Generate answer with memory extraction
    prompt = f"""You are UdaPlay, an expert video game industry assistant.

User Question: {query}

Available Information:
{context}

Instructions:
1. Provide a comprehensive, accurate answer
2. Cite your sources (database vs web)
3. Personalize based on user preferences if available
4. Identify any new facts worth remembering about this user

Generate your response."""
    
    response = llm.invoke(
        input=prompt,
        response_format=AnswerwithMemory
    )
    
    parser = PydanticOutputParser(model_class=AnswerwithMemory)
    result = parser.parse(response)
    
    print(f"[Answer] Generated response ({len(result.answer)} chars)")
    print(f"[Answer] Facts to remember: {len(result.facts_to_remember)}")
    
    return {
        "final_answer": result.answer,
        "facts_to_remember": result.facts_to_remember
    }

In [19]:
def store_memories_step(state: UdaPlayState, resource: Resource) -> Dict:
    user_id = state["user_id"]
    facts = state.get("facts_to_remember", [])
    ltm: LongTermMemory = resource.vars["long_term_memory"]
    
    stored_count = 0
    for fact in facts:
        if fact and len(fact) > 10:  # Only store meaningful facts
            memory_fragment = MemoryFragment(
                content=fact,
                owner=user_id,
                namespace="preferences"
            )
            ltm.register(memory_fragment)
            stored_count += 1
    
    print(f"[Memory] Stored {stored_count} new facts for user {user_id}")
    
    return {}

In [21]:
def create_udaplay_state_machine() -> StateMachine[UdaPlayState]:
    """Create the UdaPlay workflow state machine"""
    
    machine = StateMachine[UdaPlayState](UdaPlayState)
    
    # Create steps
    entry = EntryPoint[UdaPlayState]()
    retrieve_memories = Step[UdaPlayState]("retrieve_memories", retrieve_user_memories_step)
    rag_search = Step[UdaPlayState]("rag_search", rag_search_step)
    evaluate = Step[UdaPlayState]("evaluate", evaluate_results_step)
    web_search = Step[UdaPlayState]("web_search", web_search_step)
    generate_answer = Step[UdaPlayState]("generate_answer", generate_answer_step)
    store_memories = Step[UdaPlayState]("store_memories", store_memories_step)
    termination = Termination[UdaPlayState]()
    
    # Add all steps
    machine.add_steps([
        entry, retrieve_memories, rag_search, evaluate, 
        web_search, generate_answer, store_memories, termination
    ])
    
    # Define transitions
    machine.connect(entry, retrieve_memories)
    machine.connect(retrieve_memories, rag_search)
    machine.connect(rag_search, evaluate)
    
    # Conditional: good results → answer, bad results → web search
    def check_results_quality(state: UdaPlayState) -> Step[UdaPlayState]:
        """Decision: Use RAG results or search web?"""
        if state.get("results_are_good", False):
            return generate_answer
        return web_search
    
    machine.connect(evaluate, [generate_answer, web_search], check_results_quality)
    machine.connect(web_search, generate_answer)
    machine.connect(generate_answer, store_memories)
    machine.connect(store_memories, termination)
    
    return machine

# Create the workflow
workflow = create_udaplay_state_machine()
print("UdaPlay state machine created!")
print(f"Steps: {list(workflow.steps.keys())}")

UdaPlay state machine created!
Steps: ['__entry__', 'retrieve_memories', 'rag_search', 'evaluate', 'web_search', 'generate_answer', 'store_memories', '__termination__']


### Agent

In [26]:
class UdaplayAgent:

    def __init__(self, workflow: StateMachine[UdaPlayState], rag: RAG, long_term_memory: LongTermMemory, tavily_client: TavilyClient, llm: LLM):
        self.workflow = workflow
        self.resource = Resource(vars={
            "rag": rag,
            "long_term_memory": long_term_memory,
            "tavily": tavily_client,
            "llm": llm
        })
    def invoke(self, user_id: str, query: str) -> Run:

        print(f"UdaPlay is prcessing query for user: {user_id}")
        print(f" Query: {query}")

        initial_state: UdaPlayState = {
            "user_id": user_id,
            "user_query": query,
            "retrieved_memories": [],
            "rag_results": None,
            "rag_documents": None,
            "rag_distances": None,
            "results_are_good": False,
            "evaluation_reason": "",
            "web_search_results": None,
            "final_answer": "",
            "facts_to_remember": [],
            "total_tokens": 0
        }
        
        run= self.workflow.run(
            state= initial_state,
            resource= self.resource
        )
        return run
    def get_answer(sefl, run: Run) -> str:
        final_state= run.get_final_state()
        return final_state.get("final_answer", "No answer genrated")
    
    def get_user_memories(self, user_id:str, limit: int =10) ->List[str]:
        ltm: LongTermMemory - self.resource.vars["long_term_memory"]
        ltm: LongTermMemory = self.resource.vars["long_term_memory"]
        results = ltm.search(
            query_text="user preferences",
            owner=user_id,
            limit=limit,
            namespace="preferences"
        )
        return [frag.content for frag in results.fragments]

udaplay_agent= UdaplayAgent(
    workflow= workflow,
    rag= games_rag,
    long_term_memory= long_term_memory,
    tavily_client=tavily_client,
    llm=LLM(model="gpt-4o-mini", temperature=0.7)
)

In [30]:
run1 = udaplay_agent.invoke(
    user_id="alice",
    query="When was Pokémon Gold and Silver released?"
)

answer1 = udaplay_agent.get_answer(run1)
print("\ ANSWER:")
print(answer1)

final_state1 = run1.get_final_state()
print(f"\ Execution Stats:")
print(f"   - Steps taken: {len(run1.snapshots)}")
print(f"   - Results were good: {final_state1.get('results_are_good')}")
print(f"   - Facts stored: {len(final_state1.get('facts_to_remember', []))}")

UdaPlay is prcessing query for user: alice
 Query: When was Pokémon Gold and Silver released?
[StateMachine] Starting: __entry__
[Memory] Retrieved 1 memories for user alice
[StateMachine] Executing step: retrieve_memories
[StateMachine] Starting: __entry__
[StateMachine] Executing step: retrieve
[StateMachine] Executing step: augment
[StateMachine] Executing step: generate
[StateMachine] Terminating: __termination__
[RAG] Searched game database
[RAG] Retrieved 3 documents
[StateMachine] Executing step: rag_search
[Eval] Sufficient: True (confidence: 0.9)
[Eval] Reason: The answer directly addresses the user's question by stating the release year of Pokémon Gold and Silver. While it provides the year (1999), it does not specify the exact date or region of release, which could be considered a minor omission. Overall, the information is mostly complete and accurate.
[StateMachine] Executing step: evaluate
[Answer] Generated response (389 chars)
[Answer] Facts to remember: 1
[StateMachine

In [32]:
run2 = udaplay_agent.invoke(
    user_id="alice",
    query="Which one was the first 3D platformer Mario game?"
)

answer2 = udaplay_agent.get_answer(run2)
print("\n ANSWER:")
print(answer2)

# Check if memories were retrieved
final_state2 = run2.get_final_state()
print(f"\n Memories Retrieved: {len(final_state2.get('retrieved_memories', []))}")
for i, mem in enumerate(final_state2.get('retrieved_memories', [])[:3], 1):
    print(f"   {i}. {mem}")

UdaPlay is prcessing query for user: alice
 Query: Which one was the first 3D platformer Mario game?
[StateMachine] Starting: __entry__
[Memory] Retrieved 3 memories for user alice
[StateMachine] Executing step: retrieve_memories
[StateMachine] Starting: __entry__
[StateMachine] Executing step: retrieve
[StateMachine] Executing step: augment
[StateMachine] Executing step: generate
[StateMachine] Terminating: __termination__
[RAG] Searched game database
[RAG] Retrieved 3 documents
[StateMachine] Executing step: rag_search
[Eval] Sufficient: True (confidence: 0.95)
[Eval] Reason: The retrieved answer directly addresses the user's question by stating that 'Super Mario 64' is the first 3D platformer Mario game, and the year of release is correctly provided. The information is complete and accurate, making it sufficient to answer the question.
[StateMachine] Executing step: evaluate
[Answer] Generated response (1012 chars)
[Answer] Facts to remember: 1
[StateMachine] Executing step: generat

In [ ]:
run3 = udaplay_agent.invoke(
    user_id="alice",
    query="Was Mortal Kombat X released for Playstation 5?"
)

answer3 = udaplay_agent.get_answer(run3)
print("\n ANSWER:")
print(answer3)

final_state3 = run3.get_final_state()
print(f"\n Search Path:")
print(f"   - RAG sufficient: {final_state3.get('results_are_good')}")
print(f"   - Web search used: {final_state3.get('web_search_results') is not None}")

UdaPlay is prcessing query for user: alice
 Query: Was Mortal Kombat X released for Playstation 5?
[StateMachine] Starting: __entry__
[Memory] Retrieved 4 memories for user alice
[StateMachine] Executing step: retrieve_memories
[StateMachine] Starting: __entry__
[StateMachine] Executing step: retrieve
[StateMachine] Executing step: augment
[StateMachine] Executing step: generate
[StateMachine] Terminating: __termination__
[RAG] Searched game database
[RAG] Retrieved 3 documents
[StateMachine] Executing step: rag_search
[Eval] Sufficient: False (confidence: 0.0)
[Eval] Reason: The retrieved answer does not address the user's question regarding the release of Mortal Kombat X for Playstation 5. Additionally, there are no source documents provided to verify any information, making the response incomplete and inaccurate.
[StateMachine] Executing step: evaluate
[Web] RAG results insufficient, searching web...
[Web] Found 3 results
[StateMachine] Executing step: web_search
[Answer] Generated 